In [1]:
import evaluate
from evaluate import evaluator
from datasets import load_dataset
import transformers
import torch
from tqdm import tqdm
import pickle
import peft
from datasets import Dataset
import huggingface_hub
device = "cuda:0" if torch.cuda.is_available() else "cpu"

2025-02-22 23:13:48.215409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740262428.229733   17960 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740262428.234038   17960 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 23:13:48.249030: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
language_tested = ["en", "de", "cs", "is", "zh", "ru"] # Only from or to english
evaluated_metrics = ["bleu", "rouge", "bleurt", "sacrebleu", "comet", "meteor", "chrf", "bert_score"]
models = [
    "facebook/nllb-200-distilled-600M",
    "haoranxu/ALMA-7B",



    "meta-llama/Meta-Llama-3-8B", # TODO
    "mistralai/Mamba-Codestral-7B-v0.1", # TODO
    "mt5-base",
    "gpt2", # TODO
    "Qwen/Qwen2-0.5B", # TODO
] # TODO falcon 7B, mistral 7B, Bloom 7B, OPT 7B, MPT 7B, Bayling (?)

ds = ["haoranxu/WMT23-Test"] # TODO FLORES

## Dataset handling
We use WNT23 from the authors preprocessed split and the FLORES+ dataset, format in the same way that the WNT23 is.

### WNT23

In [2]:
ds_wnt = load_dataset("haoranxu/WMT23-Test", "en-de")["test"]
print(len(ds_wnt), ds_wnt[0:4])

557 {'en-de': [{'en': 'Police arrest 15 after violent protest outside UK refugee hotel', 'de': 'Polizei verhaftet 15 Menschen nach gewalttätigen Protesten vor einer Flüchtlingsunterkunft in Großbritannien'}, {'en': 'The incident comes after increase in numbers of refugees and asylum seekers crossing the Channel to the UK in boats. Police have arrested 15 people after an anti-refugee demonstration outside a hotel used to house asylum seekers turned violent near the English city of Liverpool. The Merseyside Police department said a police officer and two civilians sustained minor injuries during the disturbance on Friday night in Knowsley. The police force said some protesters threw objects and set a police van on fire. The people arrested, who ranged in age from 13 to 54, were detained "following violent disorder." Merseyside police commissioner Emily Spurrell told Radio City, "It was incredibly dangerous and there were a couple of injuries amongst the police officers."', 'de': 'Der Vor

### FLORES

In [27]:
from credentials import hf_token
huggingface_hub.login(token = hf_token)
ds_flores = load_dataset("openlanguagedata/flores_plus")["devtest"]

Resolving data files:   0%|          | 0/219 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/213 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/219 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/213 [00:00<?, ?it/s]

In [28]:
def transform_to_WNT_style(ds_flores, lang, lang_start="en"):
    language_to_iso = {"en": "eng", "de": "deu", "cs": "ces", "is": "isl", "zh": "zho", "ru": "rus"}
    list_sentence_lang, list_sentence_lang_start = [], []
    for elem in ds_flores:
        if elem['iso_639_3'] == language_to_iso[lang]:
            list_sentence_lang.append(elem["text"])
        elif elem['iso_639_3'] == language_to_iso[lang_start]:
            list_sentence_lang_start.append(elem["text"])
    assert len(list_sentence_lang) == len(list_sentence_lang_start)
    print(f"Number of samples: {len(list_sentence_lang)}")
    final_text_list = []
    for i in range(len(list_sentence_lang)):
        final_text_list.append({f"{lang_start}": list_sentence_lang_start[i],
                                f"{lang}": list_sentence_lang[i],})
    return Dataset.from_dict({f"{lang_start}-{lang}": final_text_list})

In [29]:
ds_flores_wnt_style = transform_to_WNT_style(ds_flores, lang="de", lang_start="en")
print(ds_flores_wnt_style[0:4])

Number of samples: 1012
{'en-de': [{'de': '„Wir haben jetzt 4 Monate alte Mäuse, die Diabetes hatten und jetzt keinen mehr haben“, fügte er hinzu.', 'en': '"We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.'}, {'de': 'Dr. Ehud Ur, Professor für Medizin an der Dalhousie University in Halifax, Nova Scotia, und Vorsitzender der Abteilung für Klinik und Wissenschaft des Kanadischen Diabetesverbands gab zu bedenken, dass die Forschungsarbeit noch in den Kinderschuhen stecke.', 'en': 'Dr. Ehud Ur, professor of medicine at Dalhousie University in Halifax, Nova Scotia and chair of the clinical and scientific division of the Canadian Diabetes Association cautioned that the research is still in its early days.'}, {'de': 'Wie einige andere Experten zeigte er sich skeptisch, ob es möglich sei, Diabetes zu heilen, und wies darauf hin, dass die Befunde für Menschen, die bereits unter Typ-1-Diabetes litten, keine Bedeutung hätten.', 'en': 'Like some other experts

## Metrics from predictions: evaluation function

In [ ]:
def evaluate_translation(sources, targets, translation_infered):
    print("Computing BLEU...")
    bleu = evaluate.load("bleu")
    results_bleu = bleu.compute(predictions=translation_infered, references=targets)

    print("Computing ROUGE...")
    rouge = evaluate.load('rouge')
    results_rouge = rouge.compute(predictions=translation_infered, references=targets)

    print("Computing BLEURT...")
    bleurt = evaluate.load("bleurt", module_type="metric")
    results_bleurt = bleurt.compute(predictions=translation_infered, references=targets)

    print("Computing SACREBLEU...")
    sacrebleu = evaluate.load("sacrebleu")
    results_sacrebleu = sacrebleu.compute(predictions=translation_infered, references=targets)

    print("Computing COMET...")
    comet_metric = evaluate.load('comet')
    results_comet = comet_metric.compute(predictions=translation_infered, references=targets, sources=sources)

    print("Computing METEOR...")
    meteor = evaluate.load('meteor')
    results_meteor = meteor.compute(predictions=translation_infered, references=targets)

    print("Computing Chrf++...")
    chrf = evaluate.load("chrf")
    results_chrf = chrf.compute(predictions=translation_infered, references=[[reference] for reference in targets])

    print("Computing_bert_score...")
    bertscore = evaluate.load("bertscore")
    results_bert = bertscore.compute(predictions=translation_infered, references=targets, lang="de")
    return {
        "bleu": results_bleu,
        "rouge": results_rouge,
        "bleurt": results_bleurt,
        "sacrebleu": results_sacrebleu,
        "comet": results_comet,
        "meteor": results_meteor,
        "chrf": results_chrf,
        "bertscore": results_bert,
    }

## Models & inference loops

### NLLB

In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
model.to(device)
equivalence_language_to_FLORES = {"en": "eng_Latn", "de": "deu_Latn", "ru": "rus_Cyrl", "is": "isl_Latn", "zh": "zho_Hans", "cz": "ces_Latn"}

In [ ]:
def get_input_targets_NLLB(dataset_wnt_format, source_lang, target_lang):
    inputs = [example[source_lang] for example in dataset_wnt_format[f"{source_lang}-{target_lang}"]]
    targets = [example[target_lang] for example in dataset_wnt_format[f"{source_lang}-{target_lang}"]]
    return inputs, inputs, targets

sources, inputs, targets = get_input_targets_NLLB(ds_wnt, source_lang="en", target_lang="de")

In [ ]:
# Translate text
def translate_list_of_str_NLLB(list_str, tokenizer, model, to_laguage):
    """
    Returns a list containing str corresponding to translation of the inputted
    """
    equivalence_language_to_FLORES = {"en": "eng_Latn", "de": "deu_Latn", "ru": "rus_Cyrl", "is": "isl_Latn", "zh": "zho_Hans", "cz": "ces_Latn"}
    with torch.no_grad():
        inputs = tokenizer(list_str, return_tensors="pt", padding=True)
        language_tgt_FLORES = equivalence_language_to_FLORES[to_laguage]
        translated = model.generate(inputs["input_ids"].to(device),
                                    forced_bos_token_id=tokenizer.convert_tokens_to_ids(language_tgt_FLORES),
                                    num_beams=5, max_length=512, early_stopping=True,
                                    ).cpu()
        translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_text

def translate_batched_NLLB(inp, model, tokenizer, target_lang, batch_size):
    """
    For 8GB VRAM, use batch_size=4
    """
    preds = []
    for i in tqdm(range(len(inp)//batch_size+1)):
        tslt = translate_list_of_str_NLLB(inp[i*batch_size : (i+1)*batch_size], tokenizer, model, target_lang)
        preds.extend(tslt)
    return preds

print(translate_list_of_str_NLLB(["Hey! I am an american", "Hallo, Ich komme aus Deutschland"], tokenizer, model, "cz"))
print(translate_list_of_str_NLLB(["Hey! I am an american", "Hallo, Ich komme aus Deutschland"], tokenizer, model, "is"))

['Jsem Američan.', 'Ahoj, jsem z Německa.']
['Ég er bandarískur.', 'Ég kom frá Þýskalandi.']


In [ ]:
predicted_trslt = translate_batched_NLLB(inputs, model, tokenizer, target_lang = "de", batch_size = 4)

100%|██████████| 140/140 [22:15<00:00,  9.54s/it]


In [ ]:
with open("./translation_NLLB_en-de_save.pkl", "wb") as f:
    pickle.dump(predicted_trslt, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("./translation_NLLB_en-de_save.pkl", "rb") as f:
    predicted_trslt = pickle.load(f)

In [ ]:
evaluation = evaluate_translation(sources, targets, predicted_trslt)

Computing BLEU...
Computing ROUGE...
Computing BLEURT...


Computing SACREBLEU...
Computing COMET...


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 18412.22it/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e66304/checkpoints/model.ckpt`
/home/dorian/miniconda3/envs/SNLP-project/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Computing METEOR...


[nltk_data] Downloading package wordnet to /home/dorian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/dorian/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/dorian/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Computing Chrf++...
Computing_bert_score...


### ALMA

In [ ]:
# Load base model and LoRA weights
model = transformers.AutoModelForCausalLM.from_pretrained("haoranxu/ALMA-7B", torch_dtype=torch.float16, device_map="auto")
tokenizer = transformers.LlamaTokenizer.from_pretrained("haoranxu/ALMA-7B", padding_side='left')
language_name = {"en": "English", "de": "German", "ru": "Russian", "is": "Islandic", "zh": "Chinese", "cz": "Czech"}

In [ ]:
def get_input_targets_ALMA(dataset, source_lang, target_lang):
    language_name = {"en": "English", "de": "German", "ru": "Russian", "is": "Islandic", "zh": "Chinese", "cz": "Czech"}
    source_lang_name = language_name[source_lang]
    target_lang_name = language_name[target_lang]
    # Use base formulation "Translate this from Chinese to English:\nChinese: 我爱机器翻译。\nEnglish:"
    sources = [example[source_lang] for example in dataset[f"{source_lang}-{target_lang}"]]
    inputs = [(
        f"Translate from {source_lang_name} to {target_lang_name}:"
        + f"\n{source_lang_name}: {example.get(source_lang)} \n{target_lang_name}:")
        for example in dataset[f"{source_lang}-{target_lang}"]]
    targets = [example[target_lang] for example in dataset[f"{source_lang}-{target_lang}"]]
    return sources, inputs, targets

sources, inputs, targets = get_input_targets_ALMA(ds, source_lang="en", target_lang="de")

In [ ]:
# Translate text
def translate_list_of_str_ALMA(list_str, tokenizer, model, target_language):
    """
    Returns a list containing str corresponding to translation of the inputted
    """
    with torch.no_grad():
        inputs = tokenizer(list_str, return_tensors="pt", padding=True)
        translated = model.generate(inputs["input_ids"].to(device),
                                    num_beams=5, max_new_tokens=5, do_sample=True,
                                    temperature=0.6, top_p=0.9
                                    ).cpu()
        translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        tgt_language_name = language_name[target_language]
        translated_text = [t.split(f"{tgt_language_name}:")[2] for t in translated_text] # Remove prompt
    return translated_text

def translate_batched_ALMA(inp, model, tokenizer, batch_size, target_language):
    """
    For 8GB VRAM, use batch_size=1
    """
    preds = []
    for i in tqdm(range(len(inp)//batch_size+1)):
        tslt = translate_list_of_str_ALMA(inp[i*batch_size : (i+1)*batch_size], tokenizer, model, target_language)
        preds.extend(tslt)
    return preds

print(translate_list_of_str_ALMA(["Translate from English to Czech:\nEnglish: Hey! I am an american \nCzech:",
                                  "Translate from German to Czech:\nGerman: Hallo, Ich komme aus Deutschland \nCzech:"],
                                  tokenizer, model, "cz"))
print(translate_list_of_str_ALMA(["Translate from English to Islandic:\nEnglish: Hey! I am an american \nIslandic:",
                                  "Translate from German to Islandic:\nGerman: Hallo, Ich komme aus Deutschland \nIslandic:"],
                                  tokenizer, model, "is"))

### Llama 3.2 Instruct

In [ ]:
## Choose between 1B and 3B model
from credentials import hf_token
huggingface_hub.login(token = hf_token)
# tokenizer = transformers.AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer = transformers.AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
# model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")
model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def get_input_targets_Llama3(dataset, source_lang, target_lang):
    language_name = {"en": "English", "de": "German", "ru": "Russian", "is": "Islandic", "zh": "Chinese", "cz": "Czech"}
    source_lang_name = language_name[source_lang]
    target_lang_name = language_name[target_lang]
    # Use base formulation "Translate this from Chinese to English:\nChinese: 我爱机器翻译。\nEnglish:"
    sources = [example[source_lang] for example in dataset[f"{source_lang}-{target_lang}"]]
    inputs = [
        [{
        "role": "system",
        "content": f"Translate from {source_lang_name} to {target_lang_name}:"
        + f"\n{source_lang_name}: {example.get(source_lang)} \n{target_lang_name}:"
        }] for example in dataset[f"{source_lang}-{target_lang}"]]
    targets = [example[target_lang] for example in dataset[f"{source_lang}-{target_lang}"]]
    return sources, inputs, targets

sources, inputs, targets = get_input_targets_Llama3(ds_wnt, source_lang="en", target_lang="de")

In [ ]:
def extract_translation_Llama3(translated_prompt):
    answer = translated_prompt.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[-1]
    translation_only = answer.split("<|end_of_text|>")[0]
    return translation_only

def translate_list_of_str_Llama3(list_str, tokenizer, model, target_language=None):
    with torch.no_grad():
        instruct_messages = tokenizer.apply_chat_template(list_str, tokenize=False, add_generation_prompt=True)
        tokens = tokenizer(instruct_messages, padding=True, return_tensors="pt")
        out_tokens = model.generate(**tokens.to(device),
                                    num_beams=5, max_new_tokens=100, do_sample=True,
                                    temperature=0.6, top_p=0.9)
        translations = tokenizer.batch_decode(out_tokens)
        translations = [extract_translation_Llama3(trans) for trans in translations]
        return translations
    
def translate_batched_Llama3(inputs, model, tokenizer, batch_size, target_language):
    """
    For 8GB VRAM, use batch_size=20 with Llama3 1B, batch_size=4 with Llama3 3B
    """
    preds = []
    for i in tqdm(range(len(inputs)//batch_size+1)):
        tslt = translate_list_of_str_Llama3(inputs[i*batch_size : (i+1)*batch_size], tokenizer, model, target_language)
        preds.extend(tslt)
    return preds

In [7]:
translate_list_of_str_Llama3(inputs[0:4], tokenizer, model)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Polizei verhaftet 15 nach gewaltsamen Protesten vor britischer Flüchtlingsunterkunft',
 'Der Vorfall kommt nach einer Zunahme der Anzahl von Flüchtlingen und Asylsuchenden, die per Boot über den Kanal nach Großbritannien gelangen. Die Polizei hat 15 Personen festgenommen, nachdem eine Demonstration gegen Flüchtlinge vor einem Hotel, in dem Asylsuchende untergebracht werden, nahe der englischen Stadt Liverpool gewaltsam geworden ist. Die Polizeidienst',
 '<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|start_header_id|>assistant\nDas Home Office nutzt das Hotel seit letzten Jahr als vorübergehende Unterkunft für Flüchtlinge, wie lokale Medien berichten. George Howarth, der Knowsley im britischen Parlament vertritt, sagte, dass die Gewalt am Freitagabend die Gemeinschaft nicht widerspiegelt. "Di

## General aggregation functions for benchmark

In [ ]:
directions = ["en-de", "de-en",
              "en-cs", "cs-en",
              "en-is", "is-en",
              "en-zh", "zh-en",
              "en-ru", "ru-en"]

def load_model(model_name):
    if model_name == "alma":
        model = transformers.AutoModelForCausalLM.from_pretrained("haoranxu/ALMA-7B", torch_dtype=torch.float16, device_map="auto")
        tokenizer = transformers.LlamaTokenizer.from_pretrained("haoranxu/ALMA-7B", padding_side='left')
        get_input_targets_fn = get_input_targets_ALMA
        tslt_fn = translate_batched_ALMA
    elif model_name == "nllb":
        tokenizer = transformers.AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
        model = transformers.AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
        model.to(device)
        get_input_targets_fn = get_input_targets_NLLB
        tslt_fn = translate_batched_NLLB
    elif model_name == "llama3-1B":
        from credentials import hf_token
        huggingface_hub.login(token = hf_token)
        tokenizer = transformers.AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
        tokenizer.pad_token = tokenizer.eos_token
        model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")
        get_input_targets_fn = get_input_targets_Llama3
        tslt_fn = translate_batched_Llama3
    elif model_name == "llama3-3B":
        from credentials import hf_token
        huggingface_hub.login(token = hf_token)
        tokenizer = transformers.AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
        tokenizer.pad_token = tokenizer.eos_token
        model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")
        get_input_targets_fn = get_input_targets_Llama3
        tslt_fn = translate_batched_Llama3
    return tokenizer, model, get_input_targets_fn, tslt_fn


def eval_translation_different_directions(directions, dataset_name, model_name, batch_size):
    # Loading full flores (if necessary)
    if dataset_name == "flores":
        from credentials import hf_token
        huggingface_hub.login(token = hf_token)
        ds_flores = load_dataset("openlanguagedata/flores_plus")["devtest"]

    # Loading corresponding model
    tokenizer, model, get_input_targets_fn, tslt_fn = load_model(model_name)

    evaluation_all_direction = {}
    for direction in directions:
        input_language, target_language = direction[0:2], direction[3:5]
        
        # Getting the right split corresponding to the translation direction
        if dataset_name == "flores":
            ds = transform_to_WNT_style(ds_flores, lang=target_language, lang_start=input_language)
        elif dataset_name == "wnt23":
            ds = load_dataset("haoranxu/WMT23-Test", direction)["test"]
        # Extracting input & targets
        sources, inputs, targets = get_input_targets_fn(ds, input_language, target_language)
        translation_pred = tslt_fn(inputs, model, tokenizer, batch_size, target_language)
        with open(f"./generated_translations/evaluations/{dataset_name}_{model_name}_{direction}.pkl", "wb") as f:
            pickle.dump(translation_pred, f, pickle.HIGHEST_PROTOCOL)
        evaluation_all_direction[f"{direction}"] = evaluate_translation(sources, targets, translation_pred)
    return evaluation_all_direction

def eval_translation_general(directions, dataset_names, model_names, batch_size):
    evaluation_all_datasets = {}
    for dataset_name in dataset_names:
        evaluation_all_models = {}
        for model_name in model_names:
            evaluation_all_models[f"{model_name}"] = eval_translation_different_directions(directions, dataset_name, model_name, batch_size)
        evaluation_all_datasets[f"{dataset_name}"] = evaluation_all_models